In [1]:
import sys
sys.path.append('../src')
from ase.io import read
vasp=read('optimized.vasp',format='vasp')
natom=len(vasp.positions)
cell=vasp.cell
positions=vasp.positions
masses=vasp.get_masses()

In [2]:
import numpy as np
nmodes=natom*3
lammps_dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))

In [3]:
structure_file='optimized.vasp'
from thermal_conductivity_AF import *
Vx,Vy,Vz=get_Vij_from_flat(structure_file,lammps_dyn)

In [4]:
#check part
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vx-GULP_Vijx) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [5]:
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vy-GULP_Vijy) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vz-GULP_Vijz) > 0.01)